In [ ]:
from keras_gym_env import ConnectFourEnv
from env_wrapper import ConnectFourSelfPLay
import numpy as np
import datetime
import tensorflow as tf
import random as rnd

from agent import DQNAgent
from buffer import Buffer
from training import train_self_play_best

# seeds
seed = 42
np.random.seed(seed)
rnd.seed(seed)

#Subfolder for Logs
config_name = "test_agent"
#createsummary writer for vusalization in tensorboard    
time_string = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# time_string = ""

best_train_path = f"logs/{config_name}/{time_string}/best_train"
best_train_writer = tf.summary.create_file_writer(best_train_path)
model_path_best = f"model/{config_name}/{time_string}/best"

# Hyperparameter
iterations = 5000
INNER_ITS = 100
BATCH_SIZE = 512
#reward_function_adapting_agent = lambda d,r: tf.where(d, tf.where(r==0.0,tf.constant(1.0),tf.constant(0.0)), r)
epsilon = 1
EPSILON_MIN = 0.01
EPSILON_DECAY = 0.995
POLYAK = 0.9

# create buffer
best_buffer = Buffer(capacity = 100000,min_size = 3000)

# create agent
env = ConnectFourEnv()
env = ConnectFourSelfPLay(ConnectFourEnv)
best_agent = DQNAgent(env,best_buffer, batch = BATCH_SIZE, model_path = model_path_best, polyak_update = POLYAK, inner_iterations = INNER_ITS)

train_self_play_best(best_agent, BATCH_SIZE, iterations, best_train_writer, epsilon= epsilon, epsilon_decay = EPSILON_DECAY,epsilon_min = EPSILON_MIN, env = env)


print("done")


2023-03-11 17:00:46.958058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 17:00:47.129280: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-11 17:00:47.679377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fkirsch/anaconda3/envs/iannwtf/lib/
2023-03-11 17:00:47.679471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_

Best Agent average reward: 0.12749003984063745


  0%|▏                                                                             | 10/5000 [03:37<30:42:51, 22.16s/it]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="logs/"

In [25]:
#Values to create a model to load_weights


from keras_gym_env import ConnectFourEnv
import numpy as np
import datetime
import tensorflow as tf

from agent import DQNAgent
from buffer import Buffer
from training import train_self_play_best

#Subfolder for Logs
config_name = "best_agent_2"
#createsummary writer for vusalization in tensorboard    
time_string = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# time_string = ""

best_train_path = f"logs/{config_name}/{time_string}/best_train"
#adapting_train_path = f"logs/{config_name}/{time_string}/adapting_train"
best_train_writer = tf.summary.create_file_writer(best_train_path)
#dapting_train_writer = tf.summary.create_file_writer(adapting_train_path)

#best_test_path = f"logs/{config_name}/{time_string}/best_test"
#adapting_test_path = f"logs/{config_name}/{time_string}/adapting_test"
#best_test_writer = tf.summary.create_file_writer(best_test_path)
#adapting_test_writer = tf.summary.create_file_writer(adapting_test_path)

#train_writer = [best_train_writer, adapting_train_writer]
#test_writer = [best_test_writer, adapting_test_writer]

model_path_best = f"model/{config_name}/{time_string}/best"
#model_path_adapting = f"model/{config_name}/{time_string}/adapting"

# Hyperparameter
iterations = 1000
INNER_ITS = 500
BATCH_SIZE = 6
reward_function_adapting_agent = lambda d,r: tf.where(d, tf.where(r==0.0,tf.constant(1.0),tf.constant(0.0)), r)
epsilon = 1 #TODO
EPSILON_DECAY = 0.99
POLYAK = 0.9

# create buffer
best_buffer = Buffer(100000,1000)
#adapting_buffer = Buffer(100000,1000)


env = ConnectFourEnv()

state = env.reset()

best_agent = DQNAgent(env,best_buffer, batch = BATCH_SIZE, model_path = model_path_best, polyak_update = POLYAK, inner_iterations = INNER_ITS)
best_agent = best_agent.model.load_weights(f'model/best_agent/20230309-155156/best/200') # small loss after 200 iterations


'<tensorflow.python.checkpoint.checkpoint.CheckpointLoadStatus object at 0x7f43081b7ac0>'

In [30]:
env = ConnectFourEnv()

state = env.reset()
player = 0
best_agent = DQNAgent(env,best_buffer, batch = BATCH_SIZE, model_path = model_path_best, polyak_update = POLYAK, inner_iterations = INNER_ITS)
best_agent = best_agent.model.load_weights(f'model/best_agent/20230309-155156/best/200') # weights for small loss after 200 iterations

newest_agent = DQNAgent(env,best_buffer, batch = BATCH_SIZE, model_path = model_path_best, polyak_update = POLYAK, inner_iterations = INNER_ITS)
newest_agent = newest_agent.model.load_weights(f'model/best_agent/20230309-155156/best/999') # newest weights


while(True):

    print("Turn Player ", player)
    env.render()
    #print(state)
    if not player:
        input_action = int(input())
    else:
        
        input_action = best_agent.select_action()

    state, r, done, info = env.step(input_action)
    
    if(done):
        print("Player ", player, " wins") if r == 1 else print("Draw")
        env.render()
        #print(state)
        break

    player = int(not player)

Turn Player  0
  ●   ●   ●   ●   ●   ●   ●  
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
  0   1   2   3   4   5   6  

 0
Turn Player  1
      ●   ●   ●   ●   ●   ●  
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
| ○ |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
  0   1   2   3   4   5   6  



AttributeError: 'CheckpointLoadStatus' object has no attribute 'select_action'